In [ ]:
import scipy.io
from os import listdir
from os.path import isfile, join
import pandas as pd

In [ ]:
from collections import defaultdict


from scipy.stats import spearmanr
from scipy.cluster import hierarchy

from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
#from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split

import numpy as np
from minepy import MINE
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
import seaborn as sns


from sklearn.dummy import DummyRegressor
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

import tensorflow as tf
import shap
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.layers import  Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
x=np.load('scalar_in.npy')
y=np.load('scalar_out_OTR2.npy')

In [ ]:
x.shape, y.shape

In [ ]:
scaler = MinMaxScaler()
scaler.fit(x)
X=scaler.transform(x)
print(np.min(X,axis=0))
print(np.max(X,axis=0))

In [ ]:
target_names=['n_particle', 'mean_gamma', 'sigma_gamma','norm_emit_x', 'norm_emit_y',
'sigma_x', 'sigma_y', 'sigma_z','sigma_px', 'sigma_py', 'sigma_pz',   
'mean_z',  'higher_order_energy_spread','cov_x__px', 'cov_z__pz', 'cov_y__py']

In [ ]:
y_target=np.log(y[:,4])
print(np.min(y_target,axis=0))
print(np.max(y_target,axis=0))
print(np.mean(y_target,axis=0))

In [ ]:
X.shape, y_target.shape

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X, y_target, test_size=0.2, random_state=42)

X_train.shape, Y_train.shape, X_val.shape, Y_val.shape

In [ ]:
temp=np.zeros((13,13))

for i in range(0,13):
    for j in range(0,13):
        mine = MINE(alpha=0.6, c=15, est="mic_approx")
        mine.compute_score(Xb[:,i],Xb[:,j])
        temp[i,j]= mine.mic()
print "done!"

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 16))
im=ax.imshow(temp,cmap="binary")
ax.set_xticks(np.arange(13))
ax.set_yticks(np.arange(13))
# ... and label them with the respective list entries
ax.set_xticklabels(feature_names);
ax.set_yticklabels(feature_names);
plt.setp(ax.get_xticklabels(), rotation=90, ha="right",rotation_mode="anchor");
cbar = ax.figure.colorbar(im, ax=ax)
#plt.show()

In [ ]:
fig, (ax1) = plt.subplots(1, 1, figsize=(24, 20))
corr_linkage = hierarchy.ward(temp)
dendro = hierarchy.dendrogram(corr_linkage, labels=feature_names, ax=ax1, leaf_rotation=90, leaf_font_size=20)
dendro_idx = np.arange(0, len(dendro['ivl']))
#ax2.imshow(corr[dendro['leaves'], :][:, dendro['leaves']])
#ax2.set_xticks(dendro_idx)
#ax2.set_yticks(dendro_idx)
#ax2.set_xticklabels(dendro['ivl'], rotation='vertical')
#ax2.set_yticklabels(dendro['ivl'])
fig.tight_layout()
ax1.grid()
#plt.show()

In [ ]:
model=Sequential()
model.add(Dense(20, activation='relu',input_shape=(13,)))
model.add(Dense(20, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1))

model.summary()

In [ ]:
# Batch size
BATCH_SIZE = 128

# Number of training epochs
EPOCHS = 5000

# Learning rate
L_RATE = 1e-5

In [ ]:
model.compile(tf.keras.optimizers.Adam(lr=L_RATE),loss='mse')

In [ ]:
%%time
history = model.fit(X_train, Y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(X_val,Y_val))

In [ ]:
model.save("CuInjSim_xemit_Model")

In [ ]:
model = tf.keras.models.load_model("CuInjSim_xemit_Model")
model.summary()

In [ ]:
ytemppred=model.predict(X_val)
Y_val.shape,ytemppred.shape

indx=np.argsort(Y_val[:])
ytemp=Y_val[indx]
predtemp=ytemppred[indx,0]

y_filtered=ytemp[::8]
pred_filtered=predtemp[::8]

xd=np.arange(pred_filtered.shape[0])

y_filtered.shape,pred_filtered.shape,xd.shape

In [ ]:
plt.figure(figsize=(8,4))
#indx=np.argsort(y_val[:,0])
plt.plot(y_filtered,'.r',label='Simulation Value')
plt.plot(pred_filtered,'ok',alpha=0.7,label='Neural Network Prediction',markersize=4)
#plt.errorbar(xd,pred_filtered,yerr=err_filtered,fmt='ok',capthick=2,capsize=2,alpha=0.2,label='BNN Standard Error')
plt.xlabel('Index')
plt.ylabel('ln($\epsilon_x$)')
plt.legend()
#plt.xlim(-2,235)
#plt.ylim(0.35,2.45)
#plt.savefig('CuInj_xemit_Model.pdf', bbox_inches='tight')

In [ ]:
background = X_train[np.random.choice(X_train.shape[0], 500, replace=False)]

In [ ]:
df_train_normed_summary = shap.kmeans(X_train, 25)

In [ ]:
explainer = shap.KernelExplainer(model.predict, df_train_normed_summary)

In [ ]:
shap_values = explainer.shap_values(X_train[:5000,:])

In [ ]:
feature_names=['distgen:r_dist:sigma_xy:value',
 'distgen:t_dist:length:value',
 'SOL1:solenoid_field_scale',
 'CQ01:b1_gradient',
 'SQ01:b1_gradient',
 'L0A_phase:dtheta0_deg',
 'L0B_phase:dtheta0_deg',
 'QA01:b1_gradient',
 'QA02:b1_gradient',
 'QE01:b1_gradient',
 'QE02:b1_gradient',
 'QE03:b1_gradient',
 'QE04:b1_gradient']

for i in range(13):
    print(i, " -- ", feature_names[i])

In [ ]:
shap.summary_plot(shap_values, features=X_train[:5000,:], plot_type="bar", show=False)

plt.xlabel("Global Importance of Feature")

In [ ]:
yp=model.predict(X_train)
tempdiff=np.abs(yp[:,0]-Y_train)
tmarker=np.where(tempdiff>0.6,1,0)

In [ ]:
t=5
plt.figure(figsize=(8,6))
plt.scatter(X_train[:5000,t],shap_values[0][:,t],s=10,c=tmarker[:5000])#X_train[:5000,2])

cbar = plt.colorbar()
#cbar.ax.set_yticklabels(['0','1','2','>3'])
#cbar.set_label('Value of Feature '+feature_names[t])
cbar.set_label('Model Accuracy Marker')

plt.xlabel("Input Feature Value "+"("+feature_names[t]+")")
plt.ylabel("Impact of Feature of Prediction")
